### Setup

In [ ]:
import numpy as np
import perceval as pcvl
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from merlin import ComputationSpace
from merlin.algorithms.kernels import FeatureMap, FidelityKernel, KernelCircuitBuilder

torch.manual_seed(0)
np.random.seed(0)

### Load data

In [ ]:
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=12
)

X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

### Fidelity kernel

In [ ]:
kernel = FidelityKernel.simple(
    input_size=4,
    n_modes=6,  
    shots=0,  
    computation_space=ComputationSpace.FOCK,
    dtype=torch.float32,
    device=torch.device("cpu"),
)

In [ ]:
K_train = kernel(X_train)
K_test = kernel(X_test, X_train)

In [ ]:
print("Train Gram shape:", K_train.shape)
print("Test Gram shape:", K_test.shape)

### Combining with scikit learn

In [ ]:
svc = SVC(kernel="precomputed")
svc.fit(K_train.detach().numpy(), y_train)
test_accuracy = svc.score(K_test.detach().numpy(), y_test)
print(f"SVM accuracy (precomputed kernel): {test_accuracy:.3f}")

In [ ]:
circuit = pcvl.Circuit(4)
circuit.add((0, 1), pcvl.BS())
circuit.add(0, pcvl.PS(pcvl.P("phi0")))
circuit.add(1, pcvl.PS(pcvl.P("phi1")))
circuit.add(2, pcvl.PS(pcvl.P("phi2")))
circuit.add(3, pcvl.PS(pcvl.P("phi3")))
circuit.add((2, 3), pcvl.BS())

experiment = pcvl.Experiment(circuit)
experiment.noise = pcvl.NoiseModel(brightness=0.93)

feature_map = FeatureMap(
    experiment=experiment,
    input_size=4,
    input_parameters="phi",
    dtype=torch.float32,
)

custom_kernel = FidelityKernel(
    feature_map=feature_map,
    input_state=[1, 0, 1, 0],
    computation_space=ComputationSpace.UNBUNCHED,
)

K_custom = custom_kernel(X_train[:20])
print("Custom kernel Gram shape:", K_custom.shape)

### Declarative kernel circuits

In [ ]:
builder = (
    KernelCircuitBuilder()
    .input_size(4)
    .n_modes(6)
    .n_photons(2)
    .angle_encoding(scale=0.7)
    .trainable(enabled=True, prefix="theta")
)

builder_kernel = builder.build_fidelity_kernel(
    input_state=[1, 1, 0, 0, 0, 0],
    shots=0,
    computation_space=ComputationSpace.FOCK,
)


In [ ]:
K_builder_train = builder_kernel(X_train)
K_builder_test = builder_kernel(X_test, X_train)

In [ ]:
print("Builder kernel Gram shape:", K_builder_train.shape)
print("Builder kernel Gram shape:", K_builder_test.shape)

In [ ]:
svc_custom_kernel = SVC(kernel="precomputed")
svc.fit(K_builder_train.detach().numpy(), y_train)
test_accuracy = svc.score(K_builder_test.detach().numpy(), y_test)
print(f"SVM accuracy (precomputed kernel): {test_accuracy:.3f}")